## what about async & await

In [1]:
async def hi():
    print("HOWDY")

In [2]:
o = hi()
o

<coroutine object hi at 0x0000016693715E08>

In [3]:
type(o)

coroutine

In [4]:
import inspect
inspect.iscoroutine(o)

True

In [5]:
inspect.iscoroutinefunction(o)

False

In [6]:
dir(o)

['__await__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'cr_await',
 'cr_code',
 'cr_frame',
 'cr_running',
 'send',
 'throw']

### send 方法启动协程
出现StopIteration异常, 由于协程内部实现类似于生成器    
为了防止异常出现，可以自己实现事件循环，或者使用asyncio实现的事件循环  
https://docs.python.org/zh-cn/3/library/asyncio-eventloop.html  

A coroutine is a consumer and an extension of a generator  
https://www.blog.pythonlibrary.org/2016/07/26/python-3-an-intro-to-asyncio/ 

In [7]:
o.send(None)

HOWDY


StopIteration: 

In [8]:
import asyncio
loop = asyncio.get_event_loop()
o = hi()
loop.run_until_complete(o)
# loop = asyncio.get_running_loop()
# loop

RuntimeError: This event loop is already running

上面报错是由于jupyter kernel也运行在一个事件循环中，这些事件循环不能嵌套使用  
https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/  

用nest_asyncio解决  
https://pypi.org/project/nest-asyncio/


In [9]:
asyncio.get_event_loop()

<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [11]:
import nest_asyncio
nest_asyncio.apply()
o = hi()
loop.run_until_complete(o)

HOWDY


In [16]:

async def sleep():
    await asyncio.sleep(5)

sl = sleep()
f = sl.send(None)
f

<Future pending>

In [17]:
type(f)

asyncio.futures.Future

In [18]:
dir(f)

['__await__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_asyncio_future_blocking',
 '_callbacks',
 '_exception',
 '_log_traceback',
 '_loop',
 '_repr_info',
 '_result',
 '_schedule_callbacks',
 '_source_traceback',
 '_state',
 'add_done_callback',
 'cancel',
 'cancelled',
 'done',
 'exception',
 'remove_done_callback',
 'result',
 'set_exception',
 'set_result']

下面的运行效果是，等待5秒后协程函数返回  
背后潜在的机制是：调用sleep() -> 调用asyncio.sleep() （异步），协程挂起-> 将控制权返回到主事件循环，主事件循环继续进行  
                 ->阻塞操作完成后(await statement finishes)，主事件循环恢复挂起的事件，继续执行，直到调用结束

In [24]:
s2 = sleep()
loop.run_until_complete(s2)  # 等待五秒结束